In [1]:
from master_dac.rld import stk

In [2]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stk_actor.agent import UnifiedSACPolicy
from stk_actor.wrappers import PreprocessObservationWrapper
import torch

from pystk2_gymnasium import AgentSpec

vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", seed=0,
                        # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                        wrapper_class=lambda x : (PreprocessObservationWrapper(x)), 
                       n_envs=1, env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"),  #'track':'abyss', #'num_kart':2, 'difficulty':0
})

# net_arch=[512,512,512]
# activation_fn=torch.nn.SiLU
# filename = 'policy_512_512_512_SiLU_2_statedict'

# net_arch=[512,512,512,512,512]
net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
# filename = 'policy_512_512_512_512_SiLU_3_statedict'
# filename = 'airl_policy_512_512_512_512_SiLU_statedict'
# filename = 'policy_normed_512_512_512_512_SiLU_statedict'
# filename = 'policy_normed_128_128_SiLU_statedict'
# filename = 'policy_normed_1024_1024_1024_Tanh_statedict'
filename = 'policy_normed_1024_1024_1024_Tanh_statedict_2'

model = PPO(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,), 
        device='cpu',
        n_steps=5,
        tensorboard_log="./outputs/"
)
action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename))
model.policy.load_state_dict(unified_policy.shared.state_dict())
# model.policy.load_state_dict(torch.load(filename))
vec_env.close()
# model = PPO.load("ppti_ppo_1500", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })
model = PPO.load("ppti_ppo4_2048_batch128_clip01_ent0001", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })
mean = torch.load('buffer_mean_2', map_location='cpu')
std = torch.load('buffer_std_2', map_location='cpu')

..:: Antarctica Rendering Engine 2.0 ::..


/Users/walidthekraken/VISA FRANCE/M2/RLD/stk_project/stk_actor/wrappers.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(mod_path/'buffer_mean_2'

Using cpu device


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_31140/2135084773.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(fi

In [3]:
unified_policy.shared.load_state_dict(PPO.load("ppti_ppo4_2048_batch128_clip01_ent0001", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), }).policy.state_dict())
torch.save(unified_policy.state_dict(), 'ppo_policy_normed_1024_1024_1024_Tanh_statedict')

In [4]:
# mean.tolist()

In [5]:
# std.tolist()

In [ ]:
# 'fortmagma',
# 'ravenbridge_mansion',
# 'snowmountain',
# 'cocoa_temple',
# 'sandtrack',    
# 'scotland', 
# 'stk_enterprise',
# 'volcano_island', # 1104
# 'xr591', # 864
from stk_actor.wrappers import StuckStopWrapper


tracks = [
    # 'black_forest',
    # 'olivermath', # 244 
    # 'minigolf', # 434 
    # 'gran_paradiso_island', # 683  
    # 'candela_city', # 521
    # 'mines', # 639
    # 'snowmountain', # 589
    # 'abyss', # 612
    # 'cornfield_crossing', # 638
    # 'hacienda', # 581
    # 'lighthouse', # 452
    # 'snowtuxpeak', # 482 
    # 'zengarden', # 356 


    'fortmagma',
    # 'ravenbridge_mansion', 
    # 'snowmountain',
    # 'cocoa_temple',
    # 'sandtrack',    
    # 'scotland', 
    # 'stk_enterprise', 
    # 'volcano_island', # 1104
    # 'xr591', # 864        
]
for track in tracks:
    vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", #seed=0,
                            # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                            wrapper_class=lambda x : (
                                StuckStopWrapper(
                                    PreprocessObservationWrapper(x, ret_dict=False, norm=False),
                                    n=128,
                                )
                            ), 
                        n_envs=1, env_kwargs={
        'render_mode':"human", 'agent':AgentSpec(use_ai=False, name="walid"), 'laps':1, 
        'track':track,
        'num_kart':15, 'difficulty':2
        })

    while True:
        obs = vec_env.reset()
        dones = [False]
        ix = 0
        while (True and not any(dones)):
            
            obs = torch.tensor(obs)
            
            obs = (obs - mean) / (std + 1e-8)
            action, _states = model.predict(obs, deterministic=False)
            obs, rewards, dones, info = vec_env.step(action)
            vec_env.render("human")
            # print(dones)
            ix += 1
            if ix == 1500:
                break
        print(track, ix)
    vec_env.close()



            




/Users/walidthekraken/VISA FRANCE/M2/RLD/stk_project/stk_actor/wrappers.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(mod_path/'buffer_mean_2'

..:: Antarctica Rendering Engine 2.0 ::..


2025-01-28 12:34:28.457 python[31146:1447953] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-28 12:34:28.457 python[31146:1447953] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/opt/homebrew/Caskroom/miniforge/base/envs/bbrl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [7]:
# import torch
# torch.save( model.policy.state_dict(), 'ppo_policy_512_512_512_512_SiLU_4_statedict')

In [ ]:
vec_env.envs[0]